Rekurzivna funkcija je funkcija, ki kliče samo sebe. Poskrbeti moramo, da se nekje ustavi.

In [4]:
def recursion(x):
    if x > 1000:
        return 1
    else:
        return x + recursion(2 * x)

print(recursion(3))

1534


In [9]:
x = [1, 2, 3, 4, 5]
# Želimo funkcijo, ke nam bo vrnila obrnjen seznam, torej [5, 4, 3, 2, 1]
# Najpreprostejša rešitev:
y = x[::1]
# Lahko pa se naloge lotimo z rekurzijo:
def r_gr(x, y=[]):
    if not x:
        return y
    else:
        y = y + x[-1:]           # Slicamo x od zadnjega elementa naprej, alternativno [x[-1]]
        return r_gr(x[:-1], y)
r_gr([1, 2, 3, 4, 5, 6, 7])

[7, 6, 5, 4, 3, 2, 1]

Ta implementacija ima manjšo težavo: ves čas ustvarjamo nove elemente, kar je potratno. Stvar se da narediti mnogo hitreje, na primer, da bi namesto celega x kot funkcija jemala le index naslednjega elementa v x. V vsakem primeru pa je stvar nekoliko počasnejša, saj se mora funkcija ponovno klicati za vsak element x, torej je globina rekurzije enaka dolžini seznama x.

In [16]:
def r_bis(x):
    n = len(x) // 2
    if n == 0:
        return x
    else:
        return r_bis(x[n:]) + r_bis(x[:n])
r_bis([1, 2, 3, 4, 5, 6, 7, 8])

[8, 7, 6, 5, 4, 3, 2, 1]

Taka metoda je hitrejša, saj je globina rekurzije logaritemska (če dolžino seznama podvojimo, bomo morali dodati le še en nivo rekurzije.)

Primer: Binomski simbol $\displaystyle{\begin{pmatrix}n \\ k\end{pmatrix}}$ predstavlja število podmnožic dolžine $k$, ki jih lahko sestavimo v množici z $n$ elementi. Stvar si lahko predstavljamo rekurzivno:
$$\displaystyle{\begin{pmatrix}n \\ k\end{pmatrix}} = \displaystyle{\begin{pmatrix}n-1 \\ k-1\end{pmatrix}} + \displaystyle{\begin{pmatrix}n-1 \\ k\end{pmatrix}}$$
$\displaystyle{\begin{pmatrix}n-1 \\ k-1\end{pmatrix}}$ predstavlja število možnosti, ki še ostanejo, če izbrani element umestimo v podmnožico, $\displaystyle{\begin{pmatrix}n-1 \\ k\end{pmatrix}}$ pa število možnosti, ki ostanejo, če ga ne.

Problem: ista vrednost se bo klicala večkrat. To lahko rešimo tako, da si vrednosti zapomnimo, lahko pa stvar rešimo tudi brez rekurzije, torej tako, da zgradimo Pascalov trikotnik.

Z rekurzijo lahko definiramo tudi integral (ne ravno integrala, temveč trapezno vsoto, ko integral razdelimo na $n$ delov).

In [17]:
def integral2(f, a, b, n=100):
    def vsota(s, x1, y1, k):
        if k>n:
            return s
        else:
            x2 = ((n - k) * a + k * b) / n
            y2 = f(x2)
            return vsota(s + (b - a) * (y1 + y2) / (2 * n), x2, y2, k+1)
    return vsota(0.0, a, f(a), 1)
integral2(lambda x: 2 ** x, 2, 4)

17.312617748651714

Repna rekurzija: Če nam uspe rekurzivno funkcijo implementirati tako, da si ti ni treba zapomniti vmesnih rešitev, je stvar bistveno hitrejša, saj zapisovanje informacij porabi precej časa. repna rekurzija za obračanje seznama bi bila:

In [18]:
def obrni_seznam(x):
    def obrni(i, y):
        if i >= len(x):
            return y
        else:
            y += [x[i]]
            return obrni(i+1, y)
    obrni(0, [])
obrni_seznam([1, 2, 3, 4, 5, 6, 7, 8])

Tu rekurzijo izvajamo tako, da se z vsakim klicem bolje približamo končni rešitvi, in na koncu vrnemo kar to.

**Preiskovanje:** Označimo četverico $(S, s, G, \sigma)$.
- $S$ predstavlja množico vseh možnih rešitev
- $s \in S$ je začetno stanje (začetni predlog za rešitev)
- G je množica ustreznih rešitev
- $\sigma$ je funkcija, ki nam na podlagi kandidata generira nove kandidate ($\sigma (s) = \{q_1, q_2, q_3, ... q_n\}$)

To pomeni, da lahko rešitev na nek problem poiščemo tako, da začnemo z $s$, pogledamo, če ni rešitev, razvijemo $\sigma (s)$ in dobimo $\{q_1, q_2 ... q_n\}$.
Nato imamo dve možnosti: Lahko najprej obravnavamo $q_1$, ga po potrebi razvijemo s $\sigma$, obravnavamo te in tako naprej. Druga možnost je, da najprej razvijemo vse $q$ in gremo šele nato naprej. Tema dvema načinoma iskanja pravimo iskanje v globino (DFS) in iskanje v širino (BFS). 

Preiskovalni algoritem izgleda približno tako:

In [19]:
s = 0 # Začetna vrednost
G = {'nekaj', None}
sigma = lambda q: sigma(q)
def preišči(S):
    front = {s}
    while front != set():
        # Izberi q iz front
        q = 'nekaj iz front'
        if q in G:
            return q
        else: front = front - q + sigma(q)
        # Paziti moramo, da je sigma taka funkcija, da se q nikoli več ne bo pojavil v front, sicer se bo funkcija zaciklala.
    return None

Primer problema: Dobimo seznam naravnih števil $x$ in želeno vsoto $v$. Želimo najti $y \subseteq x$, da bo vsota elementov $y$ enaka $v$. $\\$
V tem primeru je $S$ množica vseh podmnožic $x$, $s$ je prazen seznam, $G$ so podmnožice $x$, za katere velja $\sum _{i \in x} i = v$.
Lahko izberemo $\sigma(s) = {s, i} ~\text{ za } i \in x-s$

Boljša rešitev: Izberemo element iz $x$ in si ogledamo možnosti, da ga dodamo ali da ga ne dodamo.

In [ ]:
x = [1, 3, 5, 2, 2, 8, 3, 3, 6]
v = 10
s = []
def sigma(i, y):
    return y, y + [i]
# Tej funkciji sigma potem vnašamo elemente iz x, da dobivamo različne podmnožice. Sproti preverjamo, ali ima katera izmed njih vsoto v.